# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [13]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json
import sklearn
from azureml.automl.runtime.onnx_convert import OnnxConverter
import pandas as pd
from pandas import DataFrame

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [14]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Heart-Disease-Predection'

experiment=Experiment(ws, experiment_name)

In [15]:
#Creating Compute
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
#Create compute cluster


# choose a name for your cluster
cluster_name= "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws , name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2' , max_nodes=4)
    #create the cluster
    cpu_cluster=ComputeTarget.create(ws,cluster_name,compute_config)
    print('cpu-cluster has been created')


Found existing compute target


In [54]:
ds = Dataset.get_by_name(ws, name='Heart Failure Dataset')
data1 = ds.to_pandas_dataframe()
data1.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


In [56]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(data1, test_size=0.20,train_size=0.80)

In [78]:
from azureml.data.dataset_factory import TabularDatasetFactory

if not os.path.isdir("data"):
    os.mkdir("data")

pd.DataFrame(training_data).to_csv("data/train_data.csv")
df=ws.get_default_datastore()


df.upload(src_dir='./data', target_path='Heart Failure Dataset', overwrite=True,show_progress=True)
train_data=TabularDatasetFactory.from_delimited_files(path=df.path('Heart Failure Dataset/train_data.csv'))
print(train_data.to_pandas_dataframe())


Uploading an estimated of 2 files
Uploading ./data/test_ds.csv
Uploaded ./data/test_ds.csv, 1 files out of an estimated total of 2
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files
     Column1   age  anaemia  creatinine_phosphokinase  diabetes  \
0        132 46.00        0                       719         0   
1        167 59.00        0                        66         1   
2         36 90.00        1                        60         1   
3         67 72.00        1                       110         0   
4        197 65.00        0                       167         0   
..       ...   ...      ...                       ...       ...   
234      145 50.00        0                       185         0   
235      183 75.00        0                        99         0   
236       53 70.00        1                        69         1   
237      232 40.00        1                       129         0   
238      228

In [82]:
train_data.drop_columns('Column1')
print(train_data.to_pandas_dataframe())

     Column1   age  anaemia  creatinine_phosphokinase  diabetes  \
0        132 46.00        0                       719         0   
1        167 59.00        0                        66         1   
2         36 90.00        1                        60         1   
3         67 72.00        1                       110         0   
4        197 65.00        0                       167         0   
..       ...   ...      ...                       ...       ...   
234      145 50.00        0                       185         0   
235      183 75.00        0                        99         0   
236       53 70.00        1                        69         1   
237      232 40.00        1                       129         0   
238      228 65.00        0                        56         0   

     ejection_fraction  high_blood_pressure  platelets  serum_creatinine  \
0                   40                    1  263358.03              1.18   
1                   20                    0

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [19]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 25,
                    "max_concurrent_iterations": 4,
                    "n_cross_validations": 5,
                    "primary_metric": 'accuracy',
                    "iterations": 10,
                        
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(  
    task='classification',
    compute_target='cpu-cluster',
    training_data= train_data,
    label_column_name ='DEATH_EVENT',
    enable_onnx_compatible_models=True,
    **automl_settings
)

In [20]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_67f49fbd-8740-4cb0-a2db-8af8cf6e37b6

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardina

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [21]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [22]:
best_automl_run,fitted_model = remote_run.get_output()

In [23]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
Heart-Disease-Predection,AutoML_67f49fbd-8740-4cb0-a2db-8af8cf6e37b6_9,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [24]:
best_automl_run.get_metrics()

{'log_loss': 0.3192638709890846,
 'matthews_correlation': 0.7576469822984113,
 'recall_score_micro': 0.8913120567375886,
 'precision_score_weighted': 0.8988491391108605,
 'AUC_macro': 0.9320839866257289,
 'f1_score_micro': 0.8913120567375886,
 'average_precision_score_macro': 0.9168864824429115,
 'AUC_weighted': 0.9320839866257288,
 'accuracy': 0.8913120567375886,
 'average_precision_score_weighted': 0.9380311869662323,
 'f1_score_macro': 0.8748036342919615,
 'f1_score_weighted': 0.8914897955081665,
 'norm_macro_recall': 0.7482398449503713,
 'AUC_micro': 0.9356761795181328,
 'average_precision_score_micro': 0.9384981757831931,
 'weighted_accuracy': 0.9056387488131804,
 'recall_score_macro': 0.8741199224751857,
 'precision_score_macro': 0.8843379529231047,
 'recall_score_weighted': 0.8913120567375886,
 'precision_score_micro': 0.8913120567375886,
 'balanced_accuracy': 0.8741199224751857,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_67f49fbd-8740-4cb0-a2db-8af8cf6e37b

In [25]:
#TODO: Save the best model
best_automl_run,onnx_model =remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model,file_path="./automl_model.onnx")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [26]:
#Register the best automl model,
automl_model =remote_run.register_model(model_name = 'automl_best_model')

In [27]:
script_file_name ='inference/sscore.py'
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py','inference/sscore.py')

In [28]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_automl_run.get_environment())

In [29]:

#Local Deployment
from azureml.core.webservice import LocalWebservice
local_config = LocalWebservice.deploy_configuration(port=9001)
local_service = Model.deploy(ws, "test", [automl_model], inference_config, local_config)
local_service.wait_for_deployment(show_output=True)

#Set up the deployment_config as webservice
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True)

#Deploy the model
service = Model.deploy(
    workspace = ws,
    name = "heart-failure-model",
    models = [automl_model],
    inference_config = inference_config,
    deployment_config = aci_config, overwrite=True)

#wait until deployment is complete
service.wait_for_deployment(show_output = True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_4f3cee89203e005745d1830c04fe722a
 ---> c354c32ff2d1
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 3360e28dfab1
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImNkYmUwYjQzLTkyYTAtNDcxNS04MzhhLWYyNjQ4Y2M3YWQyMSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzNDI0NSIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzNDI0NSIsIndvcmtzcGFjZUlkIjoiY2YzYzk4NmItYzU2Ni00NjYzLTkxN2EtN2MwNWI5OGQzMzk2In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in e6f5e53e1453
 ---> 72ea4d4ee159
Step 4/5 : RUN mv '/var/azureml-app/tmpc5d9ruuh.py' /var/azureml-app/main.py
 ---> Running in 0f936624e979
 ---> aee189ad273b
Step 5/5 : 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [86]:

pd.DataFrame(test_data).to_csv("data/test_ds.csv")
test_ds=pd.read_csv("data/test_ds.csv")
test_ds.drop(test_ds.columns[0],axis=1,inplace=True)
test_ds.drop(test_ds.columns[12],axis=1,inplace=True)
test_ds



,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,70.00,0,1202,0,50,1,358000.00,0.90,141,0,0,196
1,65.00,0,157,0,65,0,263358.03,1.50,138,0,0,10
2,65.00,1,258,1,25,0,198000.00,1.40,129,1,0,235
3,70.00,0,571,1,45,1,185000.00,1.20,139,1,1,33
4,87.00,1,149,0,38,0,262000.00,0.90,140,1,0,14
5,60.00,1,260,1,38,0,255000.00,2.20,132,0,1,45
6,49.00,1,69,0,50,0,132000.00,1.00,140,0,0,147
7,50.00,0,582,1,38,0,310000.00,1.90,135,1,1,35
8,46.00,0,168,1,17,1,271000.00,2.10,124,0,0,100
9,60.00,1,588,1,60,0,194000.00,1.10,142,0,0,33


In [87]:
data=({'data':test_ds[0:3].to_dict(orient='records')})
test_sample=json.dumps(data)
output=service.run(test_sample)
print(output)

ERROR:azureml.core.webservice.aci:Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '92', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Sun, 10 Jan 2021 19:49:30 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '40e51fc0-5c6d-4143-b416-d42fca6ada79', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'Invalid input data frame: data frame has size 12 on dimension #1, while expected value is 13'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '92', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Sun, 10 Jan 2021 19:49:30 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '40e51fc0-5c6d-4143-b416-d42fca6ada79', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'Invalid input data frame: data frame has size 12 on dimension #1, while expected value is 13'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '92', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Sun, 10 Jan 2021 19:49:30 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '40e51fc0-5c6d-41

In [33]:
scoring_uri='http://d35af499-e49a-4ae5-a422-46c73d8c448f.southcentralus.azurecontainer.io/score'
key=''
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'
data = {"data":
        [
          {
            "age": 52,
            "sex": 1,
            "anaemia": 1,
            "creatinine_phosphokinase": 142,
            "diabetes": 0,
            "ejection_fraction": 40,
            "high_blood_pressure": 0,
            "platelets": 200000,
            "serum_creatinine": 1.3,
            "serum_sodium": 136,
            "smoking": 0,
            "time": 8,
          }
      ]
    }
test_sample = json.dumps(data)


resp = requests.post(scoring_uri, test_sample, headers=headers)
print(resp.json())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#Print the webservice logs
print(service.get_logs())

In [ ]:
#Delete the service
service.delete()